In [130]:
import os

import numpy as np
import tensorflow as tf
import trimesh
import tensorboard
from feature_detect.src.feat_data_keras import Data_Gen

from tensorboard.plugins.mesh import summary as mesh_summary
tf.compat.v1.disable_eager_execution()

sample_mesh = 'https://storage.googleapis.com/tensorflow-graphics/tensorboard/test_data/ShortDance07_a175_00001.ply'
log_dir = 'D:/tmp/'
batch_size = 1



In [131]:
# Camera and scene configuration.
config_dict = {
    'material': {
      'cls': 'PointsMaterial',
      'size': 1,
        # 'color':0x000fff
    }
}
config_dict2 = {
    'material': {
      'cls': 'PointsMaterial',
      'size': 1,
        # 'color':0xfff000
    }
}

# Read all sample PLY files.

In [132]:
data_gen = Data_Gen('F:/ProjectData/mesh_feature/Case-feature_npz/back')
data,  npz_name,epoch_end = data_gen.load_pkg()
vertices=data['x'][0].astype(np.float32)
features=data['y'][0].astype(np.float32)

# Add batch dimension, so our data will be of shape BxNxC.
vertices = np.expand_dims(vertices, 0)
features=np.expand_dims(features[:,1:],0)
colorsv = np.ones_like(vertices)*[0,0,255]
colorsf = np.ones_like(features)*[0,255,0]
vertices=np.concatenate([vertices,features],axis=1)
colors=np.concatenate([colorsv,colorsf],axis=1)


In [133]:
vertices_tensor = tf.compat.v1.placeholder(tf.float32, vertices.shape)
colors_tensor = tf.compat.v1.placeholder(tf.int32, colors.shape)

v_summary = mesh_summary.op(
    'v_color_tensor', 
    vertices=vertices_tensor,
    colors=colors_tensor, 
    config_dict=config_dict
)

# Create summary writer and session.
writer = tf.compat.v1.summary.FileWriter(log_dir)
sess = tf.compat.v1.Session()




In [134]:
summaries = sess.run([v_summary], feed_dict={
    vertices_tensor: vertices,
    colors_tensor: colors,
})
# Save summaries.
for summary in summaries:
  writer.add_summary(summary)

